# Syft Keras দিয়ে প্রাইভেট প্রেডিকশান  (Private Predictions with Syft Keras)

## ধাপ - ৩ঃ Syft Keras দিয়ে প্রাইভেট প্রেডিকশান - সারভিং (ক্লায়েন্ট)  (Private Prediction using Syft Keras - Serving (Client)

অভিনন্দন! আপনার মডেলটিকে সাধারণ Keras দিয়ে ট্রেইন করে এবং এটি Syft Keras দিয়ে সুরক্ষিত করার পরে, আপনি কিছু প্রাইভেট প্রেডিকশান এর রিকুয়েস্ট করতে প্রস্তুত।

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)
- Zarreen Reza - Twitter: [@zarreennreza](https://twitter.com/zarreennreza)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## ডেটা (Data)

এখানে, আমরা আমাদের MNIST ডেটাকে প্রাক-প্রক্রিয়াজাত (preprocessed) করি। এটি আমরা মডেল ট্রেইনের সময় যেভাবে প্রাক-প্রক্রিয়াজাত (preprocessed) করি সেই অনুরূপ।

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## মডেলটি সংযুক্ত করুন (Connect to model)

মডেলটি থেকে ক্যুয়েরি করার আগে মডেলটি সংযুক্ত করতে হবে। এটি করার জন্য আপনি একটি ক্লায়েন্ট তৈরি করতে পারেন। তারপর ঠিক একইরকম তিনটি TFEWorkers (alice, bob ও carol) এবং একটি ক্লাস্টার নির্ধারণ করুন। সবশেষে connect_to_model কল করুন। এটি ক্লায়েন্ট সাইডে একটি TFE সারিবদ্ধকারি (queueing)   সার্ভার তৈরি করে যা পর্ব-১৩বি এর model.serve() দ্বারা সারিবদ্ধকারি (queueing) সার্ভার সেটআপ এর সাথে সংযুক্ত হয়। এই ক্যুটি (queue)  প্লেইন টেক্সট ডেটাকে সুরক্ষিতভাবে বিভিন্ন শেয়ারে প্রেডিকশান রিকুয়েস্টে পাঠানোর দায়িত্বে থাকবে।

In [3]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [4]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## মডেল থেকে ক্যুয়েরি করা (Query model)

আপনি কিছু প্রাইভেট প্রেডিকশান পেতে প্রস্তুত! Query_model কল করার মাধ্যমে ইমেজটি উপরে তৈরি সারিতে (queue) সংযুক্ত হবে, ডেটা গোপনীয়তার সাথে  স্থানীয়ভাবে  (locally) শেয়ার হবে, এবং শেয়ারকৃত ভাগগুলো পর্ব-১৩বি এর মডেল সার্ভারে জমা দিবে।

In [5]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [6]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


অসাধারণ! আপনি এই তিনটি চিত্রকে সঠিকভাবে শ্রেণিবদ্ধ করতে সক্ষম হয়েছেন! তবে এই প্রেডিকশানগুলোর বৈশিষ্ট্য হচ্ছে এই পরিষেবাটি পাওয়ার জন্য আপনি কোনও ব্যক্তিগত তথ্য প্রকাশ করেন নি। মডেল হোস্ট আপনার ইনপুট ডেটা বা আপনার প্রেডিকশান কখনো দেখেনি এবং আপনি কখনও মডেলটি ডাউনলোড করেন নি। আপনি একটি এনক্রিপ্টেড মডেল ব্যবহার করে এনক্রিপ্টেড ডেটার উপর প্রাইভেট প্রেডিকশান পেতে সফল হয়েছেন!

এটি আমাদের নিজস্ব অ্যাপ্লিকেশনগুলিতে প্রয়োগ করার আগে, আমাদের সার্ভ করা মডেলটি মুছে ফেলতে দ্রুত পর্ব-১৩বি তে ফিরে যাই!